In [1]:
import os
import shutil

original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'valid')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir ,clss))

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: './splitted'

In [68]:
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    fnames = os.listdir(path)
    
    
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:validation_size + train_size]
    print('Validation size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)

Train size( ABBOTTS BABBLER ):  99
Validation size( ABBOTTS BABBLER ):  33
Test size( ABBOTTS BABBLER ):  33
Train size( ABBOTTS BOOBY ):  108
Validation size( ABBOTTS BOOBY ):  36
Test size( ABBOTTS BOOBY ):  36
Train size( ABYSSINIAN GROUND HORNBILL ):  107
Validation size( ABYSSINIAN GROUND HORNBILL ):  35
Test size( ABYSSINIAN GROUND HORNBILL ):  35
Train size( AFRICAN CROWNED CRANE ):  82
Validation size( AFRICAN CROWNED CRANE ):  27
Test size( AFRICAN CROWNED CRANE ):  27
Train size( AFRICAN EMERALD CUCKOO ):  92
Validation size( AFRICAN EMERALD CUCKOO ):  30
Test size( AFRICAN EMERALD CUCKOO ):  30
Train size( AFRICAN FIREFINCH ):  82
Validation size( AFRICAN FIREFINCH ):  27
Test size( AFRICAN FIREFINCH ):  27
Train size( AFRICAN OYSTER CATCHER ):  93
Validation size( AFRICAN OYSTER CATCHER ):  31
Test size( AFRICAN OYSTER CATCHER ):  31
Train size( AFRICAN PIED HORNBILL ):  112
Validation size( AFRICAN PIED HORNBILL ):  37
Test size( AFRICAN PIED HORNBILL ):  37
Train size( AL

Validation size( BLACK THROATED WARBLER ):  27
Test size( BLACK THROATED WARBLER ):  27
Train size( BLACK VENTED SHEARWATER ):  112
Validation size( BLACK VENTED SHEARWATER ):  37
Test size( BLACK VENTED SHEARWATER ):  37
Train size( BLACK VULTURE ):  93
Validation size( BLACK VULTURE ):  31
Test size( BLACK VULTURE ):  31
Train size( BLACK-CAPPED CHICKADEE ):  79
Validation size( BLACK-CAPPED CHICKADEE ):  26
Test size( BLACK-CAPPED CHICKADEE ):  26
Train size( BLACK-NECKED GREBE ):  82
Validation size( BLACK-NECKED GREBE ):  27
Test size( BLACK-NECKED GREBE ):  27
Train size( BLACK-THROATED SPARROW ):  100
Validation size( BLACK-THROATED SPARROW ):  33
Test size( BLACK-THROATED SPARROW ):  33
Train size( BLACKBURNIAM WARBLER ):  80
Validation size( BLACKBURNIAM WARBLER ):  26
Test size( BLACKBURNIAM WARBLER ):  26
Train size( BLONDE CRESTED WOODPECKER ):  86
Validation size( BLONDE CRESTED WOODPECKER ):  28
Test size( BLONDE CRESTED WOODPECKER ):  28
Train size( BLOOD PHEASANT ):  91

Validation size( CRESTED FIREBACK ):  28
Test size( CRESTED FIREBACK ):  28
Train size( CRESTED KINGFISHER ):  97
Validation size( CRESTED KINGFISHER ):  32
Test size( CRESTED KINGFISHER ):  32
Train size( CRESTED NUTHATCH ):  91
Validation size( CRESTED NUTHATCH ):  30
Test size( CRESTED NUTHATCH ):  30
Train size( CRESTED OROPENDOLA ):  98
Validation size( CRESTED OROPENDOLA ):  32
Test size( CRESTED OROPENDOLA ):  32
Train size( CRESTED SHRIKETIT ):  93
Validation size( CRESTED SHRIKETIT ):  31
Test size( CRESTED SHRIKETIT ):  31
Train size( CRIMSON CHAT ):  92
Validation size( CRIMSON CHAT ):  30
Test size( CRIMSON CHAT ):  30
Train size( CRIMSON SUNBIRD ):  118
Validation size( CRIMSON SUNBIRD ):  39
Test size( CRIMSON SUNBIRD ):  39
Train size( CROW ):  97
Validation size( CROW ):  32
Test size( CROW ):  32
Train size( CROWNED PIGEON ):  96
Validation size( CROWNED PIGEON ):  32
Test size( CROWNED PIGEON ):  32
Train size( CUBAN TODY ):  97
Validation size( CUBAN TODY ):  32
Test

Validation size( GRAY KINGBIRD ):  32
Test size( GRAY KINGBIRD ):  32
Train size( GRAY PARTRIDGE ):  96
Validation size( GRAY PARTRIDGE ):  32
Test size( GRAY PARTRIDGE ):  32
Train size( GREAT GRAY OWL ):  86
Validation size( GREAT GRAY OWL ):  28
Test size( GREAT GRAY OWL ):  28
Train size( GREAT JACAMAR ):  97
Validation size( GREAT JACAMAR ):  32
Test size( GREAT JACAMAR ):  32
Train size( GREAT KISKADEE ):  101
Validation size( GREAT KISKADEE ):  33
Test size( GREAT KISKADEE ):  33
Train size( GREAT POTOO ):  82
Validation size( GREAT POTOO ):  27
Test size( GREAT POTOO ):  27
Train size( GREAT TINAMOU ):  99
Validation size( GREAT TINAMOU ):  33
Test size( GREAT TINAMOU ):  33
Train size( GREAT XENOPS ):  89
Validation size( GREAT XENOPS ):  29
Test size( GREAT XENOPS ):  29
Train size( GREATER PEWEE ):  94
Validation size( GREATER PEWEE ):  31
Test size( GREATER PEWEE ):  31
Train size( GREATOR SAGE GROUSE ):  110
Validation size( GREATOR SAGE GROUSE ):  36
Test size( GREATOR SA

Validation size( MARABOU STORK ):  39
Test size( MARABOU STORK ):  39
Train size( MASKED BOOBY ):  79
Validation size( MASKED BOOBY ):  26
Test size( MASKED BOOBY ):  26
Train size( MASKED LAPWING ):  78
Validation size( MASKED LAPWING ):  26
Test size( MASKED LAPWING ):  26
Train size( MCKAYS BUNTING ):  112
Validation size( MCKAYS BUNTING ):  37
Test size( MCKAYS BUNTING ):  37
Train size( MIKADO  PHEASANT ):  92
Validation size( MIKADO  PHEASANT ):  30
Test size( MIKADO  PHEASANT ):  30
Train size( MOURNING DOVE ):  92
Validation size( MOURNING DOVE ):  30
Test size( MOURNING DOVE ):  30
Train size( MYNA ):  84
Validation size( MYNA ):  28
Test size( MYNA ):  28
Train size( NICOBAR PIGEON ):  92
Validation size( NICOBAR PIGEON ):  30
Test size( NICOBAR PIGEON ):  30
Train size( NOISY FRIARBIRD ):  94
Validation size( NOISY FRIARBIRD ):  31
Test size( NOISY FRIARBIRD ):  31
Train size( NORTHERN BEARDLESS TYRANNULET ):  111
Validation size( NORTHERN BEARDLESS TYRANNULET ):  37
Test si

Validation size( SANDHILL CRANE ):  31
Test size( SANDHILL CRANE ):  31
Train size( SATYR TRAGOPAN ):  102
Validation size( SATYR TRAGOPAN ):  34
Test size( SATYR TRAGOPAN ):  34
Train size( SCARLET CROWNED FRUIT DOVE ):  92
Validation size( SCARLET CROWNED FRUIT DOVE ):  30
Test size( SCARLET CROWNED FRUIT DOVE ):  30
Train size( SCARLET IBIS ):  82
Validation size( SCARLET IBIS ):  27
Test size( SCARLET IBIS ):  27
Train size( SCARLET MACAW ):  94
Validation size( SCARLET MACAW ):  31
Test size( SCARLET MACAW ):  31
Train size( SCARLET TANAGER ):  79
Validation size( SCARLET TANAGER ):  26
Test size( SCARLET TANAGER ):  26
Train size( SHOEBILL ):  105
Validation size( SHOEBILL ):  35
Test size( SHOEBILL ):  35
Train size( SHORT BILLED DOWITCHER ):  98
Validation size( SHORT BILLED DOWITCHER ):  32
Test size( SHORT BILLED DOWITCHER ):  32
Train size( SKUA ):  109
Validation size( SKUA ):  36
Test size( SKUA ):  36
Train size( SMITHS LONGSPUR ):  96
Validation size( SMITHS LONGSPUR ): 

In [3]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

BATCH_SIZE = 224
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/valid', transform=transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

cuda


In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 450)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [5]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [12]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader) 
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

AttributeError: 'Net' object has no attribute 'state'